In [1]:
import pickle
import pandas as pd
import os
import sys
sys.path
from scipy import stats
sys.path.append('../simulation')
import TS_personal_params_pooled as pp
import TS_global_params_pooled as gtp
import numpy as np

In [39]:
with open('../distributions_rl4rl/time_to_active_participants_pop_32_short_staggered_18.pkl','rb') as f:
    t = pickle.load(f)

In [3]:
seen={}
seenset=set([])
for k,v in t.items():
    for person in v:
        if person not in seenset:
            seenset.add(person)
            if k not in seen:
                seen[k]=[]
            seen[k].append(person)

In [4]:
keys = sorted(seen)
ns = {i:seen[keys[i]] for i in range(len(keys))}

In [53]:
def get_cw(t):
    first_day = {}
    keys = list(sorted(t.keys()))
   
    for time_idx in range(len(keys)):
        time = keys[time_idx]
        people=t[time]
        for person in people:
            if person not in first_day:
                first_day[person]= [time,time+pd.DateOffset(days=12*7),time+pd.DateOffset(days=12*7)-pd.DateOffset(days=7)]
    return first_day

In [54]:
cohort_to_week = get_cw(t)

In [6]:
def get_interventions(results_path,pop_size,update_time,case,phrase,ophrase):
    
    
    files = os.listdir('{}{}'.format(results_path,case))
    
    steps = []

    #check population and pop_size,update_time
    files = [f for f in files if '.DS' not in f and pop_size in f and phrase in f and ophrase not in f and 'update_days_{}'.format(update_time) in f  and '2015' not in f    ]
    #files = files[:50]
    print(len(files))
    #print(files)
    seen = 0 
    to_return={}
    to_return_o={}
    gids = {}
    for ff in files:
        #sim = int(ff.split('sim_')[1].strip('_continuous_pretreatment_base_coef_scale_hybrid.pkl'))
        #try:
        with open('{}{}/{}'.format(results_path,case,ff),'rb') as f:
            d = pickle.load(f)
            #print(d['gids'])
            #return d
            #print(d.keys())
            if len(d)>0:
            #if int(sim)<60:
                for k,v in d['history'].items():
                    #print(v.keys())
                    #print(d.keys())
                    #print(d['actions'])
                    #print(k.split('-'))
                    gid = int(k.split('-')[0])
                    rgid = int(k.split('-')[-1])
                    if gid not in to_return:
                        to_return[gid]=[]
                    if gid not in to_return_o:
                        to_return_o[gid]=[]
                    gids[gid]=rgid
                    if v['avail'] and v['decision_time']:
                        to_return[gid].append(v['action'])
                        
                        to_return_o[gid].append(v['other_action'])
               
                #print(len([k['steps'] for k in d['history'].values() if k['avail'] and k['decision_time']]))
        
    return to_return,to_return_o,gids



In [33]:
def get_interventions_by_time(results_path,pop_size,update_time,case,phrase,ophrase):
    
    
    files = os.listdir('{}{}'.format(results_path,case))
    
    steps = []

    #check population and pop_size,update_time
    files = [f for f in files if '.DS' not in f and pop_size in f and phrase in f and ophrase not in f and 'update_days_{}'.format(update_time) in f  and '2015' not in f    ]
    #files = files[:50]
    print(len(files))
    #print(files)
    seen = 0 
    to_return={}
    to_return_o={}
    gids = {}
    for ff in files:
        #sim = int(ff.split('sim_')[1].strip('_continuous_pretreatment_base_coef_scale_hybrid.pkl'))
        #try:
        with open('{}{}/{}'.format(results_path,case,ff),'rb') as f:
            d = pickle.load(f)
            #print(d['gids'])
            #return d
            #print(d.keys())
            if len(d)>0:
            #if int(sim)<60:
                for k,v in d['history'].items():
                    #print(v.keys())
                    #print(d.keys())
                    #print(d['actions'])
                    #print(k.split('-'))
                    gid = int(k.split('-')[0])
                    rgid = int(k.split('-')[-1])
                    time=pd.Timestamp(k[k.index('-')+1:-2])
        
                    if gid not in to_return:
                        to_return[gid]={}
                    if gid not in to_return_o:
                        to_return_o[gid]={}
                    gids[gid]=rgid
                    if v['avail'] and v['decision_time']:
                        to_return[gid][time]=v['action']
                        
                        to_return_o[gid][time]=v['other_action']
               
                #print(len([k['steps'] for k in d['history'].values() if k['avail'] and k['decision_time']]))
        
    return to_return,to_return_o,gids



In [7]:
def get_regret(p):
    to_return={}
    for time,dicts in p.items():
        for person,sims in dicts.items():
            if person not in to_return:
                to_return[person]={}
            for sim,r in sims.items():
                if sim not in to_return[person]:
                    to_return[person][sim]=[]
                to_return[person][sim].append(r)
                
    for person in to_return:
        for sim in to_return[person]:
            to_return[person][sim]=sum(to_return[person][sim])
    return to_return

In [61]:
test= get_interventions_by_time('../../Downloads/pooling_results_testing/personalized/','32',7,'case_two','burden825basenotreatd','inc')
#r = get_regret(t)

50


({0: {Timestamp('2015-07-27 10:00:00'): 0,
   Timestamp('2015-07-27 13:00:00'): 0,
   Timestamp('2015-07-27 17:00:00'): 0,
   Timestamp('2015-07-28 10:00:00'): 0,
   Timestamp('2015-07-28 13:00:00'): 1,
   Timestamp('2015-07-28 15:00:00'): 1,
   Timestamp('2015-07-28 17:00:00'): 1,
   Timestamp('2015-07-28 19:00:00'): 1,
   Timestamp('2015-07-29 10:00:00'): 1,
   Timestamp('2015-07-29 13:00:00'): 0,
   Timestamp('2015-07-29 15:00:00'): 1,
   Timestamp('2015-07-29 17:00:00'): 0,
   Timestamp('2015-07-29 19:00:00'): 0,
   Timestamp('2015-07-30 10:00:00'): 1,
   Timestamp('2015-07-30 15:00:00'): 0,
   Timestamp('2015-07-30 17:00:00'): 0,
   Timestamp('2015-07-31 10:00:00'): 0,
   Timestamp('2015-07-31 13:00:00'): 1,
   Timestamp('2015-07-31 17:00:00'): 1,
   Timestamp('2015-07-31 19:00:00'): 0,
   Timestamp('2015-08-01 13:00:00'): 0,
   Timestamp('2015-08-01 15:00:00'): 0,
   Timestamp('2015-08-01 17:00:00'): 1,
   Timestamp('2015-08-01 19:00:00'): 0,
   Timestamp('2015-08-02 10:00:00'): 

In [10]:
def by_cohort(cohorts,regrets,gids):
    to_return = {}
    gs = {}
    for k,v in cohorts.items():
        temp = []
        tempg=[]
        for person in v:
            reg = regrets[person]
            temp.extend(reg)
            tempg.append(gids[person])
        to_return[k]=temp
        gs[k]=tempg
    return to_return,gs

In [70]:
def by_week(person,d):
    return [v for k,v in d.items() if k<cohort_to_week[person][1] and k>=cohort_to_week[person][2]]
    

In [71]:
def by_cohort_last_week(cohorts,regrets,gids):
    to_return = {}
    gs = {}
    for k,v in cohorts.items():
        temp = []
        tempg=[]
        for person in v:
            reg = by_week(person,regrets[person])
            temp.extend(reg)
            tempg.append(gids[person])
        to_return[k]=temp
        gs[k]=tempg
    return to_return,gs

In [72]:
checkbatch = by_cohort_last_week(ns,test[0],test[2])

In [89]:

with open('../../Downloads/actions_burden825basenotreatd.pkl','rb') as f:
    tt = pickle.load(f)
checkbatch = by_cohort_last_week(ns,tt['case_one'][0],tt['case_one'][2])
#checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())

0
0.3741496598639456
1
0.28214285714285714
2
0.15789473684210525
3
0.0707070707070707
4
0.1111111111111111
5
0.06086956521739131


In [88]:
test= get_interventions_by_time('../../Downloads/pooling_results_testing/batch/','32',7,'case_one','burden825basenotreatd','inc')

checkbatch = by_cohort_last_week(ns,test[0],test[2])
#checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())

50
0
0.4897959183673469
1
0.4392857142857143
2
0.42857142857142855
3
0.494949494949495
4
0.32407407407407407
5
0.3391304347826087


In [87]:
test= get_interventions_by_time('../../Downloads/pooling_results_testing/batch/','32',7,'case_one','burden825basenotreatd','inc')

checkbatch = by_cohort_last_week(ns,test[0],test[2])
#checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())

50
0
0.4897959183673469
1
0.4392857142857143
2
0.42857142857142855
3
0.494949494949495
4
0.32407407407407407
5
0.3391304347826087


In [86]:
test= get_interventions_by_time('../../Downloads/pooling_results_testing/hob_clipped/','32',7,'case_one','burden825basenotreatd7','inc')

checkbatch = by_cohort_last_week(ns,test[0],test[2])
#checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())

50
0
0.41496598639455784
1
0.33214285714285713
2
0.41353383458646614
3
0.5252525252525253
4
0.4166666666666667
5
0.40869565217391307


In [91]:
test= get_interventions_by_time('../../Downloads/pooling_results_testing/personalized/','32',7,'case_three','burden825basenotreatd','inc')

#checkbatch = by_cohort_last_week(ns,test[0],test[2])
checktrue = by_cohort(ns,t[1],t[2])
checkbatch = by_cohort(ns,r)
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())

50


KeyError: 1

In [13]:
t= get_interventions('../../Downloads/pooling_results_testing/batch/','32',7,'case_three','burden825basenotreatd','inc')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(np.array(checktrue[0][k]).mean()-np.array(v).mean())
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
print(np.array(checks).sum())

50
0
0.5199399399399399
0.23042642642642644
1.6
-0.2895135135135135
1
0.5241324268400828
0.47853975761158735
1.3
-0.04559266922849542
2
0.4962142857142857
0.19348809523809524
1.4
-0.3027261904761905
3
0.4909545454545455
0.31707575757575757
1.5
-0.1738787878787879
4
0.48256867112100965
0.5603563474387527
1.5
0.07778767631774308
5
0.44744011976047904
0.388937125748503
1.75
-0.05850299401197606
0.9480018314267065


In [14]:
t= get_interventions('../../Downloads/pooling_results_testing/personalized/','32',7,'case_three','burden825basenotreatd','inc')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(np.array(checktrue[0][k]).mean()-np.array(v).mean())
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
print(np.array(checks).sum())

50
0
0.5534174174174175
0.2288048048048048
1.6
-0.32461261261261265
1
0.6219509311262194
0.47654744309784214
1.3
-0.14540348802837721
2
0.5242857142857142
0.19245238095238096
1.4
-0.3318333333333333
3
0.5561212121212121
0.31636363636363635
1.5
-0.23975757575757578
4
0.6525760950259837
0.5567186340014848
1.5
-0.09585746102449888
5
0.6036976047904191
0.3840868263473054
1.75
-0.21961077844311372
1.3570752491995117


In [127]:
t= get_interventions('../../Downloads/pooling_results_testing/personalized/','32',7,'case_two','burden818twotimesaveshortmajoritymellong','?')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
avs=[]
sent = []
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    sent.extend(v)
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(np.array(checktrue[0][k]).mean()-np.array(v).mean())
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
    avs.extend(checktrue[0][k])
print(np.array(checks).sum())
print(np.array(sent).mean())

50
0
0.5967825223435949
0.2755312810327706
1.6
-0.3212512413108243
1
0.6269389763779527
0.42796259842519685
1.3
-0.19897637795275586
2
0.6092892892892893
0.383003003003003
1.4
-0.22628628628628628
3
0.5917428924598269
0.3286032138442522
1.5
-0.26313967861557475
4
0.6232274459974587
0.3524015247776366
1.5
-0.27082592121982213
5
0.5726002587322122
0.22095730918499354
1.75
-0.35164294954721864
1.6321224549324818
0.6079912595598564


In [132]:
t= get_interventions('../../Downloads/pooling_results_testing/batch/','32',7,'case_three','None818twotimesaveshortmajoritysmallstaglong','?')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
avs=[]
sent = []
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    sent.extend(v)
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(np.array(checktrue[0][k]).mean()-np.array(v).mean())
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
    avs.extend(checktrue[0][k])
print(np.array(checks).sum())
print(np.array(sent).mean())

50
0
0.6173213213213213
0.5215375375375375
1.6
-0.09578378378378383
1
0.6413774756133609
0.7543245639964529
1.3
0.11294708838309198
2
0.6287142857142857
0.4380952380952381
1.4
-0.19061904761904758
3
0.6427575757575757
0.5095454545454545
1.5
-0.13321212121212123
4
0.6577579806978471
0.9226429101707498
1.5
0.26488492947290276
5
0.6536676646706587
0.7462874251497006
1.75
0.09261976047904186
0.8900667309499892
0.6394185071288789


0.5351198377581121

In [137]:
with open('../../Downloads/actions_None818twotimesaveshortmajoritysmallunstaglong.pkl','rb') as f:
    tt = pickle.load(f)
t=tt['case_two']
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
avs = []
#checkbatch = by_cohort(ns,r)
checks = []
sent=[]
for k,v in checkbatch[0].items():
    print(k)
    sent.extend(v)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(np.array(checktrue[1][k]))
    print(np.array(checktrue[0][k]).mean()-np.array(v).mean())
    avs.extend(checktrue[0][k])
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
print(np.array(checks).sum())
print(np.array(sent).mean())

0
0.4650294464075383
0.483133097762073
1.6
[2 2 2 1 1]
0.018103651354534733
1
0.5519504278548244
0.6673000885216878
1.3
[1 1 1 1 2 1 2 1 2 1]
0.11534966066686336
2
0.47155131264916467
0.5878400954653938
1.4
[1 1 1 2 2]
0.11628878281622917
3
0.5245234493192134
0.5499243570347958
1.5
[2 1 1 2]
0.025400907715582455
4
0.5176595744680851
0.5531762917933131
1.5
[1 2 1 2]
0.03551671732522799
5
0.4567143933685004
0.36646571213263
1.75
[2 2 1 2]
-0.09024868123587038
0.4009084011143081
0.5062658463832961


In [84]:
t= get_interventions('../../Downloads/pooling_results_testing/personalized/','32',7,'case_two','None818two','inc')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
avs=[]
sent=[]
for k,v in checkbatch[0].items():
    print(k)
    sent.extend(v)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]))
    print(np.array(checktrue[1][k]).mean())
    print(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
    avs.extend(checktrue[0][k])
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
print(np.array(checks).sum())
print(np.array(sent).mean())

51
0
0.562785348743603
0.4158357771260997
[2 2 2 1 1]
1.6
0.14694957161750327
1
0.6127892598480834
0.6780062795243061
[1 1 1 1 2 1 2 1 2 1]
1.3
0.06521701967622273
2
0.5877482867777507
0.5960390069688432
[1 1 1 2 2]
1.4
0.008290720191092449
3
0.5739141586867305
0.5103311673506612
[2 1 1 2]
1.5
0.06358299133606926
4
0.5883842898861672
0.5062131235472913
[1 2 1 2]
1.5
0.08217116633887589
5
0.5429544069333572
0.2991273840847939
[2 2 1 2]
1.75
0.24382702284856328
0.6100384920083268
0.5846437763896118


In [55]:
t= get_interventions('../../Downloads/pooling_results_testing/batch/','32',7,'case_two','None818','inc')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
print(np.array(checks).sum())

50
0
0.5424281524926686
0.4403284457478006
1.6
0.10209970674486801
1
0.5359197907585004
0.69521650682941
1.3
0.15929671607090967
2
0.5331756357185098
0.6212891780011828
1.4
0.08811354228267299
3
0.5301162790697674
0.5284302325581396
1.5
0.0016860465116278478
4
0.5277811550151976
0.5354863221884498
1.5
0.007705167173252203
5
0.516194086429113
0.3333737680060652
1.75
0.1828203184230478
0.5417214972063785


In [22]:
t= get_interventions('../../Downloads/pooling_results_testing/batch/','32',7,'case_three','burden817731','inc')
checkbatch = by_cohort(ns,t[0])
checktrue = by_cohort(ns,t[1])
#checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())
    print(np.array(checktrue[k]).mean())

51
0
0.6160948193285555
0.644496762801648
1
0.626173319023526
0.7852961051202989
2
0.618111149785358
0.5204316045945006
3
0.6182095205547724
0.5524513363805944
4
0.6262535548570574
0.9622960634635533
5
0.6203850909733263
0.8103397515162221


In [79]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/hob_clipped/','32',7,'case_one','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
39.89120000000001
1
40.1758
2
39.13459999999999
3
39.0105
4
36.780750000000005
5
37.34375


In [80]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/hob_clipped/','32',7,'case_two','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
37.969800000000006
1
38.65089999999999
2
37.04579999999999
3
36.618500000000004
4
33.86125
5
33.88075


In [81]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/hob_clipped/','32',7,'case_three','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
39.35122467712511
1
51.40097696795171
2
35.7128585044465
3
50.26213872618795
4
41.86718708297336
5
42.24147302469068


In [82]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/personalized/','32',7,'case_two','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
44.41660000000003
1
42.650600000000004
2
42.46820000000002
3
43.229750000000024
4
41.22400000000002
5
44.77775000000003


In [83]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/personalized/','32',7,'case_one','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
37.551800000000014
1
38.54310000000001
2
38.2934
3
38.209250000000004
4
34.943000000000005
5
36.87225000000001


In [84]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/personalized/','32',7,'case_three','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
40.16774222776907
1
51.92544741193913
2
42.09993268056138
3
54.46384500485858
4
42.634115429602744
5
42.585435238668175


In [61]:
with open('../../Downloads/regret_case_two_816747_person_dict.pkl','rb') as f:
    t = pickle.load(f)
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
21.745600000000007
1
23.173500000000008
2
24.748000000000008
3
23.859500000000004
4
22.418750000000003
5
20.159000000000002


In [62]:
with open('../../Downloads/regret_case_one_816747_person_dict.pkl','rb') as f:
    t = pickle.load(f)
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
18.397400000000005
1
19.87680000000001
2
19.95180000000001
3
21.617000000000008
4
19.92750000000001
5
22.53550000000001


In [63]:
with open('../../Downloads/regret_case_three_816747_person_dict.pkl','rb') as f:
    t = pickle.load(f)
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
20.31489657902877
1
33.41000131562465
2
21.584929740941437
3
34.48508551484087
4
31.67013433264591
5
28.544910313535265
